In [1]:
import torch
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
import numpy as np
from annoy import AnnoyIndex
import pickle

/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
checkpoint_path = '/home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch4'

In [3]:
#checkpoint_path2 = "/home/devanapalli.ravi/disk_1/paper/data/gpu_re_trained_models/lm_retrained_models/pan_india_train_threshold/pan_india_train_threshold_out_dir_5_epochs/checkpoint-375000"  


In [6]:
def load_finetuned_model(checkpoint_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModel.from_pretrained(checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
    model = model.to(device)
    print("Loading done")
    return model, tokenizer, device

model, tokenizer, device = load_finetuned_model(checkpoint_path)

Loading done


In [7]:
model_pan, tokenizer, device = load_finetuned_model(checkpoint_path)

Loading done


In [16]:
input_pickle_path = '/home/uddeshya.singh/DataSet_triplet/combined_triplets_PUNJAB_05.1.pkl'

with open(input_pickle_path, 'rb') as f:
    data = pickle.load(f)

triplets_5 = data.get('triplets', [])
addresses_5 = data.get('addresses', [])
lat_lngs_5 = data.get('lat_lngs', [])

print(f"Length of triplets: {len(triplets_5)}, {triplets_5[-1]}")
print(f"Length of addresses: {len(addresses_5)}, {addresses_5[-1]}")
print(f"Length of lat_lngs: {len(lat_lngs_5)}, {lat_lngs_5[-1]}")

Length of triplets: 3394661, (1172878, 1172877, 1172860, 3)
Length of addresses: 1172900, wardno 1 vpo sarhala ranuan Shahid Bhagat Singh Nagar District
Length of lat_lngs: 1172900, [31.2044125, 75.8583784]


In [5]:
input_pickle_path = '/home/uddeshya.singh/DataSet_triplet/combined_triplets_PUNJAB_10.pkl'

with open(input_pickle_path, 'rb') as f:
    data = pickle.load(f)

triplets = data.get('triplets', [])
addresses = data.get('addresses', [])
lat_lngs = data.get('lat_lngs', [])

print(f"Length of triplets: {len(triplets)}, {triplets[-1]}")
print(f"Length of addresses: {len(addresses)}, {addresses[-1]}")
print(f"Length of lat_lngs: {len(lat_lngs)}, {lat_lngs[-1]}")

Length of triplets: 762656, (282735, 282733, 282726, 3)
Length of addresses: 282737, vpo mandhali near guru nanak niwas gurudwara Shahid Bhagat Singh Nagar District
Length of lat_lngs: 282737, [31.191651, 75.83274809999998]


In [17]:
def get_batch_embeddings(model, tokenizer, device, texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

In [22]:
def compute_embeddings_in_batches(model, tokenizer, device, addresses, batch_size=128):
    all_embeddings = []
    for i in tqdm(range(0, len(addresses), batch_size), desc="Computing embeddings"):
        batch_texts = addresses[i:i + batch_size]
        batch_embeddings = get_batch_embeddings(model, tokenizer, device, batch_texts)
        all_embeddings.append(batch_embeddings)
    return np.vstack(all_embeddings)

In [23]:
embeddings = compute_embeddings_in_batches(model, tokenizer, device, addresses_5)

Computing embeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9164/9164 [20:54<00:00,  7.31it/s]


In [27]:
embeddings_save_path = '/home/uddeshya.singh/DataSet_triplet/embeddings_05_model1.pkl'

with open(embeddings_save_path, 'wb') as f:
    pickle.dump(embeddings, f)

print(f"Embeddings saved to {embeddings_save_path}")

with open(embeddings_save_path, 'rb') as f:
    loaded_embeddings = pickle.load(f)

print(f"Loaded embeddings shape: {loaded_embeddings.shape}")

Embeddings saved to /home/uddeshya.singh/DataSet_triplet/embeddings_05_model1.pkl
Loaded embeddings shape: (1172900, 768)


In [28]:
with open(embeddings_save_path, 'rb') as f:
    loaded_embeddings = pickle.load(f)

print(f"Loaded embeddings shape: {loaded_embeddings.shape}")

Loaded embeddings shape: (1172900, 768)


In [189]:
embeddings_pan = compute_embeddings_in_batches(model_pan, tokenizer, device, addresses_5, batch_size=256)

Computing embeddings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4582/4582 [33:20<00:00,  2.29it/s]


In [190]:
embeddings_pan.shape

(1172900, 768)

In [19]:
embeddings.shape

(282737, 768)

In [85]:
embeddings_pan.shape

(282737, 768)

In [16]:
dim = embeddings.shape[1]
annoy_index = AnnoyIndex(dim, 'angular')

for i, embedding in tqdm(enumerate(embeddings), total=len(embeddings), desc="Adding items to AnnoyIndex"):
    annoy_index.add_item(i, embedding)

annoy_index.build(10)


True

In [17]:
def find_nearest_neighbors(model, tokenizer, device, annoy_index, addresses, lat_lngs, query_address, num_neighbors=5):
    query_embedding = get_batch_embeddings(model, tokenizer, device, [query_address])[0]
    nearest_neighbors = annoy_index.get_nns_by_vector(query_embedding, num_neighbors, include_distances=True)
    neighbors_info = []
    for idx in nearest_neighbors[0]:
        neighbor_info = {
            "address": addresses[idx],
            "coordinates": lat_lngs[idx]
        }
        neighbors_info.append(neighbor_info)
    return neighbors_info

In [20]:
query_address = "164 f ghal kalan moga Moga"
coor = [30.8128771, 75.10901820000002]
nearest_neighbors_info = find_nearest_neighbors(model, tokenizer, device, annoy_index, addresses, lat_lngs, query_address, num_neighbors=10)
print("Nearest neighbors:")
for neighbor in nearest_neighbors_info:
    print(f"Address: {neighbor['address']}, Coordinates: {neighbor['coordinates']}")


Nearest neighbors:
Address: sardar ajit singh near chotti masjid Barnala District, Coordinates: [30.4282837, 75.38305140000001]
Address: h no d 6 435 new abadi near elementary school kot khalsa amritsar street titu sweet shop Amritsar, Coordinates: [31.6227159, 74.8304383]
Address: vpo usman shahid tehsil dasuya aminder singh so gurmit singh dasua subdistrictmacdonal Hoshiarpur District, Coordinates: [31.830058899999997, 75.65322859999998]
Address:  18 sant fateh singh ngr dugri rd ldh ludhiana punjab india Ludhiana, Coordinates: [30.879886699999997, 75.8451422]
Address:  27263 street no 2 hari nagar lal singh basti Bathinda, Coordinates: [30.1972566, 74.9319392]
Address: h 2322 c block ranjit avenue c block market block c market ranjit avenue Amritsar, Coordinates: [31.659080699999997, 74.8658094]
Address: plot no 17 gujrat bhawan sector 24 c chandigardh daskhin marg opp batra cinma Chandigarh, Coordinates: [30.741028600000003, 76.7574027]
Address: street no 4 baba fateh singh nagar s

In [22]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
from sklearn.neighbors import NearestNeighbors
import time
from geopy.distance import geodesic as vincenty

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [40]:
! pip install geopy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: http://jfrog.fkinternal.com/artifactory/api/pypi/python_virtual/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 1.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.1 MB/s eta 0:00:00


In [164]:
knn_ann = NearestNeighbors(n_neighbors=3)

In [29]:
print('ji')

ji


In [165]:
print('starting now ', time.strftime('%X %x %Z'))

knn_ann.fit(embeddings)

print('done now ', time.strftime('%X %x %Z'))

starting now  18:15:09 06/25/24 IST
done now  18:15:09 06/25/24 IST


In [129]:
print('starting now ', time.strftime('%X %x %Z'))

knn.fit(embeddings_pan)

print('done now ', time.strftime('%X %x %Z'))

starting now  18:02:47 06/25/24 IST
done now  18:02:47 06/25/24 IST


In [27]:
embeddings.shape

(282737, 768)

In [124]:
k = knn.kneighbors(embeddings[101].reshape(1, -1))

In [125]:
k

(array([[6.74349565e-07, 1.77979612e+00, 1.89073455e+00]]),
 array([[  101, 41859, 90477]]))

In [126]:
addresses[101]

'safu wala adda palvir karyana storedaroli bhai daroli bhai Moga District'

In [127]:
addresses[41859]

' 234 mukerian subdistrict Hoshiarpur District'

In [128]:
addresses[90477]

'vpo babri jiwanwal GURDASPUR'

In [177]:
k = knn_ann.kneighbors(embeddings[16204].reshape(1, -1))

In [178]:
k

(array([[0.        , 1.41673064, 1.90807331]]), array([[16204, 21069, 20860]]))

In [179]:
addresses[16204]

' 28 4 shivalik officers enclave mamun cantt Pathankot'

In [180]:
addresses[21069]

'p 17 2 shivalik officers enclave mamun cantt Pathankot'

In [181]:
addresses[20860]

'norpur tcp patankot patankot norpur tcp mob 9815664509  Pathankot'

In [43]:
def get_deviation_for_df(coor_1, coor_2):
    try:
        return vincenty(coor_1,coor_2).meters
    except Exception as e:
        print("exception occured ", e)
        return None

In [161]:
get_deviation_for_df(lat_lngs[50000], lat_lngs[16753])
                     

204050.01374227175

In [183]:
get_deviation_for_df(lat_lngs[16204], lat_lngs[21069])
                     

9.439000335286295

In [182]:
get_deviation_for_df(lat_lngs[16204], lat_lngs[20860])
                     

3389.8195310909455

In [25]:
import json
import time
import torch
import pickle
import pandas as pd
import numpy as np
from h3 import h3
from geopy.distance import geodesic as vincenty
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from sklearn.neighbors import NearestNeighbors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
def load_finetuned_model(checkpoint_path):
    model = AutoModel.from_pretrained(checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    return model, tokenizer, device


def get_centroid_of_grid(grid_id_local):
    try:
        return h3.h3_to_geo(grid_id_local)
    except Exception as e:
        print("Exception occurred", grid_id_local, e)
        return None

def get_deviation_for_df(lat_local, lng_local, centroid_local):
    try:
        return int(vincenty((lat_local, lng_local), centroid_local).meters)
    except Exception as e:
        print("Exception occurred ", e)
        return None

def get_drift_for_row(row_local):
    """Get drift between predicted coordinates and lookup coordinates"""
    pred_coor = get_centroid_of_grid(row_local['top_grid'])
    pred_drift = get_deviation_for_df(row_local["lookup_lat"], row_local["lookup_lng"], pred_coor)
    return pred_drift

def get_accuracy_for_df(drift_list_local):
    accuracy_dict_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for i in drift_list_local:
        if i is None or pd.isnull(i):
            accuracy_dict_local["None"] += 1
            continue

        if i < 100:
            accuracy_dict_local[100] += 1
        elif i < 200:
            accuracy_dict_local[200] += 1
        elif i < 500:
            accuracy_dict_local[500] += 1
        elif i < 1000:
            accuracy_dict_local[1000] += 1
        elif i < 2000:
            accuracy_dict_local[2000] += 1
        else:
            accuracy_dict_local["rest"] += 1

    accuracy_dict_local[200] += accuracy_dict_local[100]
    accuracy_dict_local[500] += accuracy_dict_local[200]
    accuracy_dict_local[1000] += accuracy_dict_local[500]
    accuracy_dict_local[2000] += accuracy_dict_local[1000]

    accuracy_dict_per_local = {100: 0, 200: 0, 500: 0, 1000: 0, 2000: 0, "rest": 0, "None": 0}
    for k, v in accuracy_dict_local.items():
        accuracy_dict_per_local[k] = float(v) / max(1, (accuracy_dict_local[2000] + accuracy_dict_local["rest"]))

    final_accuracy_dict_local = {'num': accuracy_dict_local, 'per': accuracy_dict_per_local}
    return final_accuracy_dict_local

def get_batch_embeddings(model, tokenizer, device, texts):
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

def compute_embeddings_in_batches(model, tokenizer, device, addresses, batch_size=128):
    all_embeddings = []
    for i in tqdm(range(0, len(addresses), batch_size), desc="Computing embeddings"):
        batch_texts = addresses[i:i + batch_size]
        batch_embeddings = get_batch_embeddings(model, tokenizer, device, batch_texts)
        all_embeddings.append(batch_embeddings)
    return np.vstack(all_embeddings)


def get_nearest_neighbors_and_medoid(embeddings, knn, query_embedding, k=3):
    distances, indices = knn.kneighbors(query_embedding.reshape(1, -1), n_neighbors=k)
    neighbors = embeddings[indices[0]]

    medoid_index = np.argmin(np.sum(distances, axis=1))
    medoid_embedding = neighbors[medoid_index]

    return medoid_embedding, indices[0]


def embedding_to_grid_id(embedding):

    return "placeholder_grid_id"


embeddings_save_path = '/home/uddeshya.singh/DataSet_triplet/embeddings.pkl'
with open(embeddings_save_path, 'rb') as f:
    embeddings = pickle.load(f)


file_path_test = '/disk_4/uddeshya/Dataset/combined_cleaned_triplet_test_PUNJAB_full_1.csv'
test_df = pd.read_csv(file_path_test)

checkpoint_path = '/home/uddeshya.singh/Experiments/punjab_roberta_model/checkpoint_epoch_1'
model, tokenizer, device = load_finetuned_model(checkpoint_path)


knn = NearestNeighbors(n_neighbors=3)
knn.fit(embeddings)


test_df['top_grid'] = None 
pred_grid_list = []
for index, row in test_df.iterrows():
    query_address = row['cleaned_address']

    query_embedding = get_batch_embeddings(model, tokenizer, device, [query_address])[0]


    medoid_embedding, neighbor_indices = get_nearest_neighbors_and_medoid(embeddings, knn, query_embedding, k=3)


    top_grid_id = embedding_to_grid_id(medoid_embedding)
    pred_grid_list.append(top_grid_id)

test_df['top_grid'] = pred_grid_list

# Calculate drift and accuracy
test_df['pred_drift'] = test_df.apply(lambda row: get_drift_for_row(row), axis=1)
test_drift_acc_dict = get_accuracy_for_df(test_df.pred_drift.tolist())

print(test_drift_acc_dict)
